In [1]:
import requests
from bs4 import BeautifulSoup
from bs4.element import Tag
import feedparser
import urllib
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pickle
import pandas as pd

In [2]:
keyword = "kohli"

In [3]:
def CNN_Scraper(max_pages):
    base = "https://edition.cnn.com/"
    #browser = webdriver.Chrome('C:/chromedriver_win32/chromedriver.exe')
    browser = webdriver.Chrome(ChromeDriverManager().install())
    load_content = browser.implicitly_wait(30)
    base_url = 'https://edition.cnn.com/search?q=' + keyword + '&sort=newest&category=sport&size=100'
    
    CNN_title   = []
    CNN_date   = []
    CNN_article   = []
    article_count = 0
        

    #iterate over pages and extract   
    
    for page in range(1, max_pages + 1):
        url= base_url + "&page=%d&from=%d" % (page, article_count)
        browser.get(url)
        load_content
        soup = BeautifulSoup(browser.page_source,'lxml')
        search_results = soup.find('div', {'class':'cnn-search__results-list'})
        contents = search_results.find_all('div', {'class':'cnn-search__result-contents'})

        for content in contents:
            try:
                title = content.find('h3').text
                link = content.find('a')
                link_url = link['href']    

                date = content.find('div',{'class':'cnn-search__result-publish-date'}).text.strip()
                article = content.find('div',{'class':'cnn-search__result-body'}).text
            except:
                continue
            CNN_title.append(title)
            CNN_date.append(date)
            CNN_article.append(article)
            
        article_count += 100   
        
    #Arranging as data frame
    
    df = pd.DataFrame()
    df['title'] = CNN_title
    df['date'] = CNN_date      
    df['article'] = CNN_article 
    return df        

    #print("Complete")

    browser.quit()
    
CNN_data = CNN_Scraper(2)
#CNN_data.to_pickle("CNN_data")



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/97.0.4692.71/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\srini\.wdm\drivers\chromedriver\win32\97.0.4692.71]
<ipython-input-3-cd59821b00fb>:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install())


In [15]:
CNN_data.to_csv('samp.csv')